# Pandas와 PyMySQL 연동

In [ ]:
!pip install PyMySQL

In [2]:
import pymysql
import pandas as pd

host_name = 'localhost' # 구글 컴퓨터라 내 컴퓨터의 localhost에 접속 불가 - 어디서나 접속할 수 있는 원격 DB가 필요하다
host_port = 3306
username = 'root'
password = ''
database_name = 'fisa'

db = pymysql.connect(
    host=host_name,     # MySQL Server Address
    port=host_port,          # MySQL Server Port
    user=username,      # MySQL username
    passwd=password,    # password for MySQL username
    db=database_name,   # Database name
    charset='utf8'
)


- 클라우드에 설치된 MySQL 8.0 SERVER


In [ ]:
!pip install pymysql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pymysql
import pandas as pd
import warnings  # 인터프리터가 내는 모든 경고메시지를 출력하지 않기
warnings.simplefilter(action='ignore') # Ignore warnings


host_name = ?
host_port = 3306
username = ?
password = ?
database_name = ?

conn = pymysql.connect(
    host=host_name,     # MySQL Server Address
    port=host_port,          # MySQL Server Port
    user=username,      # MySQL username
    passwd=password,    # password for MySQL username
    db=database_name,   # Database name
    charset='utf8mb4'
)

- pymysql의 연결 객체를 이용하여 pandas의 자료형인 데이터프레임 객체 생성하기

In [3]:
import pandas as pd
df = pd.read_sql("select * from emp",conn) # read_csv 하면 csv 파일을 불러왔듯이 reqd_sql 함수
df

NameError: name 'conn' is not defined

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   empno     14 non-null     int64  
 1   ename     14 non-null     object 
 2   job       14 non-null     object 
 3   mgr       13 non-null     float64
 4   hiredate  14 non-null     object 
 5   sal       14 non-null     float64
 6   comm      4 non-null      float64
 7   deptno    14 non-null     int64  
dtypes: float64(3), int64(2), object(3)
memory usage: 1.0+ KB


In [ ]:
pd.read_sql("show tables",conn)

,Tables_in_fisa
0,dept
1,emp
2,emp_new
3,salgrade
4,titanic_raw


- 접속정보는 중요하므로 별개 파일에 저장 (주로 json이나 yaml 사용)

In [ ]:
%%writefile db.yaml
HOST: ?
USER: ?
PASSWD: ?

Overwriting db.yaml


In [5]:
import yaml
DB_INFO = "../db.yaml"
with open(DB_INFO,"r") as f:
    db_info = yaml.load(f, Loader=yaml.Loader)

- DB 접속 정보

In [6]:
db_info

{'LOCAL_HOST': 'localhost',
 'LOCAL_PORT': 3306,
 'LOCAL_USER': 'root',
 'LOCAL_PASS': '',
 'CLASS_HOST': '118.67.131.22',
 'CLASS_PORT': 3306,
 'CLASS_USER': 'fisaai',
 'CLASS_PASS': 'woorifisa3!W'}

In [7]:
HOST = db_info["LOCAL_HOST"]
USER = db_info["LOCAL_USER"]
PASSWD = db_info["LOCAL_PASS"]
PORT = 3306

In [8]:
import pymysql

conn = pymysql.connect(
    user = USER,
    passwd = PASSWD,
    host = HOST,
    port = PORT,
    db = 'fisa'
)
conn

In [9]:
pd.read_sql_query("show tables",conn)


/var/folders/3p/5k6m1fh905j6r_ch2rqt3y700000gn/T/ipykernel_3133/2971206481.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query("show tables",conn)


,Tables_in_fisa
0,box_office
1,box_office_two
2,dept
3,dept01_v
4,emp
5,emp01
6,emp01_v
7,emp05
8,emp_
9,emp_dept_v


In [10]:
df = pd.read_sql_query("select * from emp;",conn) # read_sql_table, read_sql_query
df

# df = pd.read_sql_table("emp",conn) # 향후 사용할 타 프레임워크와 호환되는 함수. pymysql에서는 작동하지 않음

/var/folders/3p/5k6m1fh905j6r_ch2rqt3y700000gn/T/ipykernel_3133/3258271060.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("select * from emp;",conn) # read_sql_table, read_sql_query


,empno,ename,job,mgr,hiredate,sal,comm,deptno
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20
1,7499,AAALLLLLEN,SALESMAN,7698.0,1981-02-20,1700.0,300.0,30
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30
5,7698,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,NaN,30
6,7782,CLARK,MANAGER,7839.0,1981-06-09,2450.0,NaN,10
7,7788,SCOTT,ANALYST,7566.0,1987-04-19,3000.0,NaN,20
8,7839,KING,PRESIDENT,NaN,1981-11-17,5000.0,NaN,10
9,7844,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30


In [11]:
db.close() ### 이전 DB 커넥션을 끊어주세요

# DF를 데이터베이스 서버로 보내기

## SQLAlchemy
- python에서 사용하는 대표적인 ORM
- ORM(Object Relational Mapping) 이란?
    - 객체와 DB의 테이블이 매핑을 이루는 것을 말한다.
    - DB의 테이블 객체화 시켜서 데이터를 CRUD
    - SQL 을 직접 작성하지 않고 테이블을 조작할수 있다.
    - 사용하는 DBMS가 변경된다면 엔진만 바꿔주면 된다.
    - 쿼리 대신 메소드를 이용해서 CRUD 한다.

``` df.to_sql(name=테이블이름, con=engine, if_exists='append', index=False)```
-  Dataframe은 항상 index가 있기 때문에, 테이블 구조와 안맞을 수 있음, 그래서 index=False 로 작성
-  if_exists = 'fail' : 같은 이름의 Table이 존재할 경우 ValueError 가 남
   if_exists = 'replace': 같은 이름의 Table이 존재할 경우 기존 Table을 Drop하고 새로운 값을 Insert함
  if_exists = 'append': 같은 이름의 Table이 존재할 경우 기존 Table에 추가로 새로운 값을 Insert함
- 각 column name을 테이블의 컬럼명과 동일하게 하면 해당 컬럼에 데이터 입력
- empno 는 PRIMARY KEY로 AUTO_INCREMENT 옵션을 넣었으므로, 데이터 입력을 하지 않음

In [14]:
!conda install sqlalchemy -y
!conda install pymysql -y

Channels:
 - defaults
 - conda-forge
Platform: osx-arm64
Solving environment: done

## Package Plan ##

  environment location: /opt/homebrew/Caskroom/miniconda/base/envs/0731

  added / updated specs:
    - sqlalchemy


The following NEW packages will be INSTALLED:

  greenlet           pkgs/main/osx-arm64::greenlet-3.0.1-py312h313beb8_0 
  sqlalchemy         pkgs/main/osx-arm64::sqlalchemy-2.0.30-py312h80987f9_0 




Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [15]:
import pymysql
from sqlalchemy import create_engine  # InnoDB
import pandas as pd


## 방법은 거의
## 접속할DB종류+접속에사용할패키지명 + :// + userid:password @ 주소:포트번호/db?인코딩방법 의 형태를 띔
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWD}@{HOST}:{PORT}/fisa?charset=utf8mb4")

# local csv file 을 mysql table로 변환하고 DataBase에 저장
df = pd.read_csv("../00_settings/jjangu_list2.csv")
# name : table명, con : 연결할 엔진, if_exists : 기존 테이블이 있을 경우 어떻게 할지 정하는 옵션, index : 인덱스를 함께 저장할지 여부
df.to_sql(name="jjangu", con=engine, if_exists='append', index=False)

8

In [17]:
# engine에 연결된 DB에서 jjangu 테이블을 불러와서 보여줌
df = pd.read_sql_table("jjangu",engine)
df

,Unnamed: 0,번호,이름,등록일자,테스트점수,단위,담당,반
0,0,1,짱구,2022-03-11,60,점,나미리,장미
1,1,2,짱아,2022-03-12,67,점,채성아,개나리
2,2,3,철수,2022-03-13,100,점,나미리,장미
3,3,4,맹구,2022-03-14,70,점,나미리,장미
4,4,5,유리,2022-03-15,75,점,나미리,장미
5,5,6,훈이,2022-03-15,78,점,나미리,장미
6,6,7,짱구아빠,2022-03-16,66,점,채성아,개나리
7,7,8,짱구엄마,2022-03-17,85,점,채성아,개나리


In [ ]:
# df1 이라는 이름으로 등록일자는 datetime, 테스트점수는 float으로 자료형을 변경한 후에
# jjanggu1 이라는 테이블에 to_sql 명령어를 사용해서 데이터를 넣어주세요. 자료형을 확인해보세요.
# db.yaml에 있는 정보를 바탕으로 중요정보를 익명처리해주세요.
import pymysql
from sqlalchemy import create_engine  # InnoDB
import pandas as pd


## 방법은 거의
## 접속할DB종류+접속에사용할패키지명 + :// + userid:password @ 주소:포트번호/db?인코딩방법 의 형태를 띔
engine = create_engine(
    f"mysql+pymysql://{USER}:{PASSWD}@{HOST}:{PORT}/fisa?charset=utf8mb4"
)
df = pd.read_csv("jjangu_list2 (1).csv", index_col=0)
df

df.to_sql(name="jjanggu", con=engine, if_exists="append", index=False)

- DB 선택 없이 접속

In [18]:
conn = pymysql.connect(
    user = USER,
    passwd = PASSWD,
    host = HOST,
    port = PORT,
    # autocommit = True # default : False # 오토커밋 기능 설정
)
conn

In [19]:
conn.autocommit_mode

False

- DB 선택하기

In [20]:
conn.select_db("fisa") # use DB명;

- 동적 SQL 사용

In [ ]:
# SQL문 실행 - %s 로 변하는 값이 들어갈 자리를 비워둠.


# 비워둔 자리 개수만큼 2번째 인자로 값을 튜플 안에 순서대로 전달


# 데이타 Fetch


- df를 csv 파일로 저장

- 사용후엔 꼭 connection을 닫아주세요!

In [ ]:
conn.close()